In [1]:
# pip install datasets

In [2]:
from datasets import load_dataset
news = load_dataset('multi_news', split = 'test')

c:\Users\Abhi\anaconda3\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [4]:
train_news = news.train_test_split(test_size = .2)

In [5]:
# pip install transformers accelerate 

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')

In [8]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized = train_news.map(preprocess_function , batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForSeq2Seq , AutoModelForSeq2SeqLM , Seq2SeqTrainingArguments, Seq2SeqTrainer

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer , model = 't5-small')

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

In [13]:
training_args = Seq2SeqTrainingArguments(
output_dir="./results",
evaluation_strategy="epoch",
learning_rate=2e-5,
per_device_train_batch_size=10,
per_device_eval_batch_size=10,
weight_decay=0.01,
save_total_limit=3,
num_train_epochs=10,
# fp16=True,
)

In [14]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized['train'],
    eval_dataset = tokenized['test'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

c:\Users\Abhi\anaconda3\Lib\site-packages\accelerate\accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: abhishekupadhyay9336 (abhishekupadhyay9336-amazon). Use `wandb login --relogin` to force relogin


  0%|          | 0/4500 [00:00<?, ?it/s]

In [16]:
def pred(document):
  device = model.device
  tokenized= tokenizer([document], return_tensors = 'pt')
  print(tokenized)
  tokenized ={k:v.to(device) for k , v in tokenized.items()}
  results = model.generate(**tokenized, max_length = 128)
  results = results.to('cpu')
  pred = tokenizer.decode(results[0])
  return pred

In [17]:
document = "National Archives Yes, it's that time again, folks. It's the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you're here, why don't you sign up to follow us on Twitter. Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth. The unemployment rate dipped, but mostly because more Americans stopped looking for work. The Labor Department says the economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2 percent, the lowest since January 2009. The rate dropped because fewer people searched for jobs. The official unemployment tally only includes those seeking work. The economy has added 858,000 jobs since December _ the best four months of hiring in two years. But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending."
human_summary = """– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today's jobs report. Reaction on the Wall Street Journal's MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment."""

In [18]:
summary = pred(document)

{'input_ids': tensor([[  868, 18499,  2163,     6,    34,    31,     7,    24,    97,   541,
             6,  5265,     5,    94,    31,     7,     8,   166,  1701,    13,
             8,   847,     6,   116,    21,    80,   664,    18,     7,    32,
            18, 25572,   798,     8,  3984,    13,  3556,  1887,     6,  2386,
            11,  5140,  1887,    33,    66,  7901,    15,    26,    30,    80,
           589,    10, 15106,     5,    71,  1434,  2270,    30,     8,   412,
             5,   134,     5,  4311,  1419,    21,  1762,  8046,     8,  4107,
             7,    44, 23899,     3,     9,     5,    51,     5,   368,  1060,
            97,  1772,    80,    13,     8,   167,   359, 23052,     7,    30,
           149,     8,  2717,   623,    15,    26,   383,     8,  1767,   847,
             5, 19539,  1628,    33,    21,   460, 11212,   126,  2476,    12,
            36,   990,     6,  1315,    12, 20163,     7,  5492,    15,    26,
            57, 21236,  6193,  3529, 1

In [19]:
summary

"<pad><extra_id_0>,<extra_id_1>,<extra_id_2>,<extra_id_3> the show.<extra_id_4>,<extra_id_5>,<extra_id_6> the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. It's the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs.</s>"